In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#importing the data and overview 

train_df= pd.read_csv('../input/titanic/train.csv')
test_df= pd.read_csv('../input/titanic/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df['Survived'].value_counts()

# survived = 1
# didn't survive = 0

In [ ]:
#EDA

sns.countplot(data=train_df, x='Survived')

# survived = 1
# didn't survive = 0

In [ ]:
sns.boxplot(data=train_df, x='Survived', y='Age')

# survived = 1
# didn't survive = 0

In [ ]:
sns.scatterplot(data=train_df, x='Sex', y='Age', hue='Survived')

In [ ]:
sns.heatmap(train_df.corr(), annot=True)

In [ ]:
#Data preparation
#looking for null values and taking care of them

train_df.isnull().sum()

In [ ]:
train_df["Age"].fillna(train_df["Age"].mean(), inplace = True)

In [ ]:
train_df['Sex']=train_df['Sex'].replace('male', 0)
train_df['Sex']=train_df['Sex'].replace('female', 1)

In [ ]:
train_df.drop(['Name', 'PassengerId', 'Fare', 'Ticket','Embarked', 'Cabin'], axis = 1, inplace = True)

In [ ]:
train_df.isnull().sum()

In [ ]:
missing=train_df.isnull().sum().sort_values(ascending=False)
missing=missing.drop(missing[missing==0].index)
missing

In [ ]:
#defining features and the label

X= train_df.drop('Survived', axis=1)
y= train_df['Survived']

In [ ]:
#spliting the dataset to work on train and test


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#feature scaling


from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
scaler.fit(X_train)

In [ ]:
scaled_X_train= scaler.transform(X_train)
scaled_X_test= scaler.transform(X_test)

In [ ]:
#training the model

from sklearn.neighbors import KNeighborsClassifier

knn_model= KNeighborsClassifier(n_neighbors=1)
knn_model.fit(scaled_X_train, y_train)

In [ ]:
#predicting test

y_pred= knn_model.predict(scaled_X_test)

In [ ]:
#predicted value VS actual value of test data

pd.DataFrame({'Y_Test':y_test, 'Y_Pred': y_pred})

In [ ]:
#testing the model

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
#Elbow method

testing_error_amount= []


for k in range (1, 30):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(scaled_X_train, y_train)
    
    y_pred_test = knn_model.predict(scaled_X_test)
    
    test_error=1- accuracy_score(y_test, y_pred_test)
    testing_error_amount.append(test_error)

In [ ]:
testing_error_amount

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 30), testing_error_amount, label='Test Error')
plt.legend()
plt.ylabel('Error Amount')
plt.xlabel('K Value')

In [ ]:
#pipeline

scaler= StandardScaler()

knn= KNeighborsClassifier()
knn.get_params().keys()

In [ ]:
operations= [('scaler', scaler), ('knn', knn)]

In [ ]:
from sklearn.pipeline import Pipeline

pipe= Pipeline(operations)


from sklearn.model_selection import GridSearchCV

k_values= list(range(1, 20))
param_grid= {'knn__n_neighbors': k_values}
full_cv_classifier= GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
full_cv_classifier.fit(X_train, y_train)

In [ ]:
full_cv_classifier.best_estimator_.get_params()

In [ ]:
full_cv_classifier.cv_results_.keys()

In [ ]:
# finalize the model

scaler= StandardScaler()
knn19= KNeighborsClassifier(n_neighbors=19)
operations= [('scaler', scaler), ('knn19', knn19)]

pipe= Pipeline(operations)

pipe.fit(X_train, y_train)

In [ ]:
pipe_pred= pipe.predict(X_test)

In [ ]:
print(classification_report(y_test, pipe_pred))